# Statistics about WL similarities per iteration

Following are statistics about concept maps WL similarities, ie. how many matches there have been per iteration.

In [ ]:
from notebook_prelude import *

In [ ]:
data = {}

In [ ]:
for file in glob('data/CACHE/*splitted*.npy'):
    filename = file.split('/')[-1]
    if filename in data: continue
    with open(file, 'rb') as f:
        res = pickle.load(f)
    assert len(res) == 6
    data[filename] = res

In [ ]:
sum_data = collections.defaultdict(dict)
for file, res in helper.log_progress(data.items()):
    phi_train, phi_test, X_train, X_test, Y_train, Y_test = res
    num_iterations = len(phi_train)
    assert len(phi_train) == len(phi_test)
    print(file)
    for h, (phi_te, phi_tr) in enumerate(zip(phi_test, phi_train)):
        assert phi_te.shape[1] == phi_tr.shape[1]
        sum_data[file][h] = phi_te.dot(phi_tr.T).sum()
        gc.collect()
        print('\th={}'.format(h))

In [ ]:
gc.collect();

In [ ]:
df = pd.DataFrame(sum_data).unstack().to_frame('gram_sum').unstack()
for a, b in [(1, 0), (2, 1)]:
    df['ratio_{}_{}'.format(a, b)] = df['gram_sum', a] / df['gram_sum', b]
df['same_label'] = df.index.str.contains('same-label')
df['dataset'] = df.index.map(filename_utils.get_dataset_from_filename)
df.set_index(['dataset', 'same_label'], inplace=True)
df